[View in Colaboratory](https://colab.research.google.com/github/saitota/ProgrammingKnock100/blob/master/Chapter04_Q30_Q39.ipynb)

# :muscle: 前書き
詳細は[第1章](https://qiita.com/saitotak/items/73852ecb7cc4b10b42a5)参照、引き続き言語処理100本ノックの第3章を回答していきます。本章では TBD


# :muscle: 第4章: 形態素解析
> 夏目漱石の小説『吾輩は猫である』の文章（[neko.txt](http://www.cl.ecei.tohoku.ac.jp/nlp100/data/neko.txt)）をMeCabを使って形態素解析し，その結果をneko.txt.mecabというファイルに保存せよ．このファイルを用いて，以下の問に対応するプログラムを実装せよ．
> なお，問題37, 38, 39は[matplotlib](http://matplotlib.org/)もしくは[Gnuplot](http://www.gnuplot.info/)を用いるとよい．

UTF-8,LF の1万行程度の日本語プレーンテキストでした。形態素解析と統計のライブラリを存分に使うことになりそうな感じです。

```text:neko.txt
一

　吾輩は猫である。
名前はまだ無い。

　どこで生れたかとんと見当がつかぬ。
何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。
（以下略）
```

## ⚾ 30. 形態素解析結果の読み込み
> 形態素解析結果（neko.txt.mecab）を読み込むプログラムを実装せよ．ただし，各形態素は表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をキーとするマッピング型に格納し，1文を形態素（マッピング型）のリストとして表現せよ．第4章の残りの問題では，ここで作ったプログラムを活用せよ．


## 下準備として
- ファイルダウンロード
- janomeのインストール

In [4]:
import urllib.request
urllib.request.urlretrieve('http://www.cl.ecei.tohoku.ac.jp/nlp100/data/neko.txt','./neko.txt')


('./neko.txt', <http.client.HTTPMessage at 0x7fe57ccb29e8>)

In [5]:
!pip install janome

### 解0-1 janomeによる形態素解析と、neko.txt.janome のテキスト作成（おそい）
- MeCab ではなく janome をインストールします、なぜならインストールが簡単だったから。
- `janome.tokenizer` を使って形態素解析します
    - [janome package — janome.tokenizer](http://mocobeta.github.io/janome/api/janome.html#module-janome.tokenizer)
- やたら遅かったので、開始・終了 時間を計測するコードを入れました
- 実測：42sec


In [6]:
from janome.tokenizer import Tokenizer
import janome
import datetime

print('start:' + str(datetime.datetime.now()))

t = Tokenizer()

# WriteのたびにOpen
with open ('./neko.txt','r',encoding='utf-8') as file_temp:
    for line in file_temp:
        # 形態素解析して保存
        for token in t.tokenize(line, stream=True):
            with open ('./neko.txt.janome-01','a',encoding='utf-8') as file_janome:
                    file_janome.write(str(token) + '\n')

print('end:' + str(datetime.datetime.now()))

'''
一	名詞,数,*,*,*,*,一,イチ,イチ
吾輩	名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
猫	名詞,一般,*,*,*,*,猫,ネコ,ネコ
で	助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ
ある	助動詞,*,*,*,五段・ラ行アル,基本形,ある,アル,アル
。	記号,句点,*,*,*,*,。,。,。
名前	名詞,一般,*,*,*,*,名前,ナマエ,ナマエ
（以下略）
'''


start:2018-05-08 12:42:38.687158
end:2018-05-08 12:43:20.010313


'\n一\t名詞,数,*,*,*,*,一,イチ,イチ\n吾輩\t名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ\nは\t助詞,係助詞,*,*,*,*,は,ハ,ワ\n猫\t名詞,一般,*,*,*,*,猫,ネコ,ネコ\nで\t助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ\nある\t助動詞,*,*,*,五段・ラ行アル,基本形,ある,アル,アル\n。\t記号,句点,*,*,*,*,。,。,。\n名前\t名詞,一般,*,*,*,*,名前,ナマエ,ナマエ\n（以下略）\n'

### 解0-2 高速化
- メモリ枯渇を鑑み 形態素解析→write を繰り返すようにしてみましたが、やたらと時間がかかったため、一括で書き込むようにして高速化を図ります
- 実測：29sec

In [7]:
print('start:' + str(datetime.datetime.now()))

t = Tokenizer()

# Openしてから
with open ('./neko.txt','r',encoding='utf-8') as file_temp , open ('./neko.txt.janome-02','a',encoding='utf-8') as file_janome:
    for line in file_temp:
        # 形態素解析して保存
        for x in t.tokenize(line, stream=True):
            file_janome.write(str(x) + '\n') 
            

print('end:' + str(datetime.datetime.now()))

start:2018-05-08 12:43:20.715597
end:2018-05-08 12:43:49.633588


### 解0-3 さらに高速化
- それでも遅い気がしたので、書き込み部分をリスト内包表記にしました、多少早くなった
- 実測：27sec


In [8]:
print('start:' + str(datetime.datetime.now()))

t = Tokenizer()

# 一部リスト内包表記
with open ('./neko.txt','r',encoding='utf-8') as file_temp , open ('./neko.txt.janome-03','a',encoding='utf-8') as file_janome:
    for line in file_temp:
        # 形態素解析して保存
        [file_janome.write(str(x) + '\n') for x in t.tokenize(line, stream=True)]
            

print('end:' + str(datetime.datetime.now()))

start:2018-05-08 12:43:50.426822
end:2018-05-08 12:44:17.077403


### 解0-4 さらに高速化・・・失敗
- 読みにくいですがぜんぶリスト内包表記にしました
- 実測：88sec
  - 遅くなった何故。。。

In [11]:
print('start:' + str(datetime.datetime.now()))

t = Tokenizer()

# リスト内包表記
with open ('./neko.txt','r',encoding='utf-8') as file_temp , open ('./neko.txt.janome-04','a',encoding='utf-8') as file_janome:
    [
        [
            file_janome.write(str(x) + '\n')
            for x in t.tokenize(y, stream=True)
        ]
        for y in file_temp
    ]

print('end:' + str(datetime.datetime.now()))

start:2018-05-08 12:54:00.055594
end:2018-05-08 12:54:26.359976


### 解1 neko.txt.janome のテキスト作成
-  形態素はタブ区切り、品詞等の分類はカンマ区切りなので成型します

In [28]:
with open ('./neko.txt.janome-04','r',encoding='utf-8') as file_temp:
    text = file_temp.readlines()

lst_janome=[]
for line in text:
    lst_janome.append(
        { 'surface': line.split('\t')[0], # 表層形
          'pos'    : line.split('\t')[1].split(',')[0], # 品詞
          'pos1'   : line.split('\t')[1].split(',')[1], # 品詞細分類1
          'base'   : line.split('\t')[1].split(',')[4]   # 基本形
        })

for i in range(10):
    print(lst_janome[i])
  

{'surface': '一', 'pos': '名詞', 'pos1': '数', 'base': '*'}
{'surface': '吾輩', 'pos': '名詞', 'pos1': '代名詞', 'base': '*'}
{'surface': 'は', 'pos': '助詞', 'pos1': '係助詞', 'base': '*'}
{'surface': '猫', 'pos': '名詞', 'pos1': '一般', 'base': '*'}
{'surface': 'で', 'pos': '助動詞', 'pos1': '*', 'base': '特殊・ダ'}
{'surface': 'ある', 'pos': '助動詞', 'pos1': '*', 'base': '五段・ラ行アル'}
{'surface': '。', 'pos': '記号', 'pos1': '句点', 'base': '*'}
{'surface': '名前', 'pos': '名詞', 'pos1': '一般', 'base': '*'}
{'surface': 'は', 'pos': '助詞', 'pos1': '係助詞', 'base': '*'}
{'surface': 'まだ', 'pos': '副詞', 'pos1': '助詞類接続', 'base': '*'}



## ⚾ 31. 動詞
> 動詞の表層形をすべて抽出せよ．

In [0]:
with open ('./neko.txt.janome','r',encoding='utf-8') as file_temp:
    #for line in file_temp:
    #    print(line)
    text = file_temp.readlines()

for line in text:
    print(line)

TBD

## ⚾ 32. 動詞の原形
> 動詞の原形をすべて抽出せよ．

## ⚾ 33. サ変名詞
> サ変接続の名詞をすべて抽出せよ．

## ⚾ 34. 「AのB」
> 2つの名詞が「の」で連結されている名詞句を抽出せよ．

## ⚾ 35. 名詞の連接
> 名詞の連接（連続して出現する名詞）を最長一致で抽出せよ．

## ⚾ 36. 単語の出現頻度
> 文章中に出現する単語とその出現頻度を求め，出現頻度の高い順に並べよ．

## ⚾ 37. 頻度上位10語
> 出現頻度が高い10語とその出現頻度をグラフ（例えば棒グラフなど）で表示せよ．

## ⚾ 38. ヒストグラム
> 単語の出現頻度のヒストグラム（横軸に出現頻度，縦軸に出現頻度をとる単語の種類数を棒グラフで表したもの）を描け．

## ⚾ 39. Zipfの法則
> 単語の出現頻度順位を横軸，その出現頻度を縦軸として，両対数グラフをプロットせよ．
